A number of positions of objects that appear to be plausibly variable also appear to be offset from the stars that are probably actually variable. This tends to happen a lot in the neighborhood between two bright-ish stars. It is probably because the initial screening algorithm tries to grab the position with the maximum variation, rather than the brightest, in order to capture dim stars in the limbs of bright stars. It may also be because the star that is actually varying is too bright for our initial upper limit cut on source brightness (to avoid large numbers of artifacts in bright stars).

A few of the obvious-looking flare stars are also slightly offset from center of the aperture. Let's try to automatically fix those, too.

In [13]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import pyarrow.parquet as pq
import pandas as pd
from rich import print
import warnings
import os

In [2]:
def angularSeparation(ra1, dec1, ra2, dec2):

    d2r = np.pi/180.
    ra2deg = 1./d2r

    d1 = dec1*d2r
    d2 = dec2*d2r

    r1 = ra1*d2r
    r2 = ra2*d2r

    a = np.sin((d2-d1)/2.)**2.+np.cos(d1)*np.cos(d2)*np.sin((r2-r1)/2.)**2.
    r = 2*np.arcsin(np.sqrt(a))

    return r*ra2deg

In [3]:
header_data = pd.read_csv('../ref/mislike_image_header_table.csv')
catalog_filename = '../ref/catalog_nd_daostarfinder.parquet'
catalog_file = pq.ParquetFile(catalog_filename)

In [4]:
#adjust_gifs = !ls /Users/cm/GFCAT/gfcat/test_gifs/sorted/adjust/*
adjust_gifs = !ls /Users/cm/GFCAT/gfcat/test_gifs/sorted/flares_adjust/*
print(f"{len(adjust_gifs)} source positions to be adjusted to the nearest brighter source.")

24 source positions to be adjusted to the nearest brighter source.

In [5]:
#expt = header_data.loc[header_data['ECLIPSE']==eclipse].loc[header_data['BAND']=='NUV']['EXPT_0'].values[0]

In [16]:
for adjust_gif in adjust_gifs:
    eclipse = int(adjust_gif.split('/')[-1].split('-')[0][1:])
    obj_id = int(adjust_gif.split('/')[-1].split('-')[1])
    #print(eclipse)
    this_star = pq.read_table(catalog_filename,filters =
                              [('eclipse','=',eclipse),
                               ('obj_id','=',obj_id)]).to_pandas()
    #print(this_star[['obj_id','ra','dec']].values[0])
    ra,dec, = this_star[['ra','dec']].values[0]
    this_area = pq.read_table(catalog_filename,filters =
                                [('eclipse','=',eclipse),
                                 #('obj_id','=',obj_id),
                                 ('ra','>=',ra-0.02),
                                 ('ra','<=',ra+0.02),
                                 ('dec','>=',dec-0.02),
                                 ('dec','<=',dec+0.02),
                                 ('aperture_sum_n_12_8','>',this_star['aperture_sum_n_12_8'].values[0])]
                             ).to_pandas()
    try:
        ix = np.argmin(angularSeparation(ra,dec,
                      this_area['ra'].values,
                      this_area['dec'].values))
    except ValueError:
        print(f'No alternative nearby source for e{eclipse}-{obj_id}')
        continue
    print('python make_gfcat.py',int(eclipse),
          int(this_area.iloc[ix]['obj_id']),f'# {int(obj_id)}')#,
          #float(this_area.iloc[ix]['ra']),
          #float(this_area.iloc[ix]['dec']))

python make_gfcat.py 3858 613803858 # 614703858

python make_gfcat.py 4028 404804028 # 404904028

No alternative nearby source for e7940-555607940

No alternative nearby source for e9709-1055909709

python make_gfcat.py 13533 3364113533 # 3357413533

...

No alternative nearby source for e42499-766342499

python make_gfcat.py 43238 1534643238 # 1530943238

No alternative nearby source for e43828-1063943828

python make_gfcat.py 44684 3066544684 # 3062344684

No alternative nearby source for e45629-1000545629

In [7]:
this_area

,obj_id,ra,dec,xcenter,ycenter,eclipse,sharpness,roundness1,roundness2,npix,...,aperture_sum_edge_n_51_2,aperture_sum_f_12_8,aperture_sum_mask_f_12_8,aperture_sum_edge_f_12_8,aperture_sum_f_25_6,aperture_sum_mask_f_25_6,aperture_sum_edge_f_25_6,aperture_sum_f_51_2,aperture_sum_mask_f_51_2,aperture_sum_edge_f_51_2


### Even after automated readjustment, a number of sources are still off and will be manually adjusted by figuring out their positions in ds9

In [22]:
os.listdir("/Users/cm/GFCAT/gfcat/test_known_jpg/sorted/")

['marginal',
 '.DS_Store',
 'eclipse',
 'trend',
 'moving',
 'artifact',
 'unk variable',
 'flare']

In [36]:
for fn in (os.listdir("/Users/cm/GFCAT/gfcat/test_gifs/sorted/adjust2/") +
           os.listdir("/Users/cm/GFCAT/gfcat/test_gifs/sorted/flares_adjust/")):
    #print(f"aws s3 cp s3://dream-pool/{fn.split('-')[0]}/{fn.split('-')[0]}-nd-full.fits.gz .")
    #print(fn.split('-')[0],fn.split('-')[1])
    continue

### manually revised positions for a small number of objects
#### //eclipse  objid     ra      dec
e06882 2182706882 310.03898  -0.86965

e15112 583415112  173.68275  45.53918

e29166 1128729166 355.72871   1.65753

e29390 1588929390  10.92245 -42.34763

e29643 965529643   53.06506 -29.51007

e43201 738043201  217.15856  57.75332

e03858 614703858  135.96049  52.35710

e04028 404904028  189.00277  62.49956

e07940 555607940    6.43415  17.08139

e09709 1055909709 164.46273  -3.76613

e13533 3357413533  16.65372  45.38989

e13656 613113656   24.76303 -17.94913

e16497 1520916497 219.53099  33.10083

e18081 80418081   358.46939 -10.54433

e18380 3958418380   7.70821 -71.84528

e18753 958718753   51.26207 -17.21407

e23505 1597023505  13.11492   0.09855

e26488 1218326488 212.80185   6.83090

e28619 1333228619 326.15289   0.70565

e30122 549930122   53.00912 -27.04739

e30668 1439130668 133.73297   2.57409

e31274 1708031274 120.70829  36.45803

e36705 1284736705 146.54298  15.54300

e40521 1328540521  29.19705  -0.35408

e40930 3656640930  70.63517  18.43965

e42499 766342499  165.10157  12.73207

e43238 1530943238 243.41123  54.66503

e43828 1063943828 338.34099 -60.93026

e44684 3062344684 308.13204   7.97132

e45629 1000545629 160.64052  76.02885